In [109]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
import itertools
import csv

In [65]:
def load_data(filepath):
    return pd.read_parquet(filepath)

In [66]:
def save_parameters_to_csv(params, file_path):
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=params.keys())
        writer.writeheader()
        writer.writerow(params)
    print(f'Parameters successfully saved to {file_path}' )

In [67]:
def save_metrics_to_csv(metrics, file_path):
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=metrics.keys())
        writer.writeheader()
        writer.writerow(metrics)
    print(f'Metrics saved to {file_path}')

In [68]:
# Extract necessary features for model input
def create_features(df):
    df['week'] = df['trip_start_timestamp'].dt.isocalendar().week
    df['2_hour_window'] = df['trip_start_timestamp'].dt.floor('2h').dt.hour
    return df

In [69]:
def prepare_weather_metrics(df):
    df['Temperature'] = df['Temperature'].str.extract('(\d+)').astype(float)
    df['Wind Speed'] = pd.to_numeric(df['Wind Speed'].str.extract('(\d+)')[0], errors='coerce')
    df['Humidity'] = df['Humidity'].str.replace(r'[^\d%]', '', regex=True).str.replace('%', '').astype(float) / 100
    df['Precip.'] = df['Precip.'].replace('Trace', 0).replace(r'[^\d.]', '', regex=True).astype(float)
    return df

In [70]:
# Aggregate and encode data for model training
def create_prediction_dataset(df, temporal_res, spatial_res):
    # Define time window based on temporal resolution
    time_window_col = f'{temporal_res}_hour_window' if temporal_res < 24 else 'weekday'

    # Define grouping columns dynamically
    grouping_columns = [f'h3_res{spatial_res}_pickup', 'week', time_window_col]

    # Common aggregation dictionary
    aggregations = {
        'Temperature': 'mean',
        'Humidity': 'mean',
        'Precip.': 'mean',
        'Wind Speed': 'mean',
        'trip_id': 'count'
    }

    # Group and aggregate data
    df_grouped = df.groupby(grouping_columns).agg(aggregations).reset_index()
    df_grouped.rename(columns={'trip_id': 'demand'}, inplace=True)

    # Encode categorical variable
    df_encoded = pd.get_dummies(df_grouped, columns=[f'h3_res{spatial_res}_pickup'], prefix=f'h3_res{spatial_res}')

    # Print dataset information
    print(f'\nDataset prepared with \nspatial resolution: h3_res_{spatial_res} \ntemporal resolution: {temporal_res}_h\n')

    return df_encoded

### Attempt to Embed H3 Resolutions
In an effort to refine our data pre-processing steps and potentially improve the performance of the model, we explored an alternative to the traditional one-hot coding method for processing geographic H3 resolutions. Specifically, we experimented with embedding the H3 resolution indices using a neural network to capture more complex spatial relationships in a more compact representation.

Despite the theoretical advantages of this method, our implementation encountered significant operational problems. The training was inefficient and exhibited a lack of convergence, meaning that the training process could not converge within a reasonable time frame.

As a result, this approach was not included in our pipeline and we decided to revert to one-hot coding for the H3 resolutions.

In [71]:
# h3_index_res7 = df_demand['h3_res7_pickup'].astype('category').cat.codes.values
# num_h3_indices_res7 = np.max(h3_index_res7) + 1
#
# # Neural network for training embeddings for Resolution 7
# embedding_dim = 10
# model_res7 = Sequential([
#     Embedding(input_dim=num_h3_indices_res7, output_dim=embedding_dim, input_length=1),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(1)
# ])
# model_res7.compile(optimizer='adam', loss='mean_squared_error')
# y_res7 = df_demand['demand_1_hour_window_h3_res7_pickup'].values
# model_res7.fit(h3_index_res7, y_res7, epochs=5, batch_size=32, validation_split=0.2)
#
# # Extract embeddings for Resolution 7
# embeddings_res7 = model_res7.layers[0].get_weights()[0]
# for i in range(embedding_dim):
#     df_demand[f'h3_res7_embedding_{i}'] = embeddings_res7[h3_index_res7, i]


In [72]:
def split_scale_data(df):
    X = df.drop('demand', axis=1)
    y = df[['demand']]

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X)
    y_train_scaled = scaler.fit_transform(y).ravel()
    X_test_scaled = scaler.fit_transform(X_test)
    y_test_scaled = scaler.fit_transform(y_test).ravel()

    return X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled,scaler

In [73]:
def gridsearch(X_train,y_train, temporal_res, spatial_res, hyperparameters):
    model = SVR()

    print('Starting grid search...')
    grid_search = GridSearchCV(
        estimator = model,
        param_grid = hyperparameters,
        scoring = 'neg_root_mean_squared_error',
        verbose = 3,
    )
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_

    model_path = f'data/models/model_spatial_{spatial_res}_temporal_{temporal_res}.joblib'
    dump(best_model, model_path)

    params_path = f'data/models/parameter_spatial_{spatial_res}_temporal_{temporal_res}.csv'
    save_parameters_to_csv(best_params, params_path)

    print(f'Best parameters: \n{best_params}')
    return best_model

In [74]:
def model_evaluation(model, scaler, X_test, y_test, temporal_res, spatial_res):
    y_pred = model.predict(X_test)

    mse = mean_squared_error(scaler.inverse_transform(y_test.reshape(-1, 1)),scaler.inverse_transform(y_pred.reshape(-1, 1)))
    rmse = float(np.sqrt(mse))
    r2 = r2_score(scaler.inverse_transform(y_test.reshape(-1, 1)),scaler.inverse_transform(y_pred.reshape(-1, 1)))

    evaluation_dict = {
        'mse' : mse,
        'rmse' : rmse,
        'r2' : r2,
    }

    metrrics_path = f'data/models/metrics_spatial_{spatial_res}_temporal_{temporal_res}.csv'
    save_metrics_to_csv(evaluation_dict, metrrics_path)
    print(f'Evaluation metrics: \n{evaluation_dict}')


### Model Training Approach

Due to the computationally intensive nature of our model training processes, we have decided to initially train all models on just 10% of the available data. This approach allows us to efficiently evaluate different models and their parameters without incurring the high computational cost of full dataset training.

After this preliminary phase, we will identify the most promising model—based on performance metrics and parameter optimization—on this smaller dataset. Subsequently, we will proceed to train this model on the entire dataset. This step ensures that we leverage the most effective model configuration while optimizing our resources and time during the experimental phase.

This strategy strikes a balance between computational efficiency and model accuracy, allowing us to iterate quickly and refine our approach based on preliminary results before committing to more resource-intensive training sessions.

In [75]:
df = load_data('data/prepped/weather_taxi_merged_df.parquet')
df = create_features(df)
df = prepare_weather_metrics(df)

spatial_res = [7,8]
temporal_res = [1,2,6,24]

hyperparameters = [
    {
        'kernel': ['linear'],
        'epsilon': [0.1, 0.001],
        'C': [1,10]  # Fixed 'C' value for 'linear' kernel
    },
    {
        'kernel': ['rbf'],
        'epsilon': [0.1, 0.001],
        'C': [1,10]  # Fixed 'C' value for 'rbf' kernel
    },
    {
        'kernel': ['poly'],
        'epsilon': [0.1, 0.001],
        'C': [1, 10],  # Varying 'C' values for 'poly' kernel
        'degree': [2, 3]  # 'degree' only applies to 'poly' kernel
    }
]

for spatial_res, temporal_res in itertools.product(spatial_res, temporal_res):
    df_pred = create_prediction_dataset(df, temporal_res, spatial_res)
    df_pred = df_pred.sample(frac=0.1)
    X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled,scaler = split_scale_data(df_pred)
    model = gridsearch(X_train_scaled,y_train_scaled, temporal_res, spatial_res, hyperparameters)
    model_evaluation(model, scaler, X_test_scaled, y_test_scaled, temporal_res, spatial_res)


Dataset prepared with 
spatial resolution: h3_res_7 
temporal resolution: 1_h

Starting grid search...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ..C=1, epsilon=0.1, kernel=linear;, score=-0.601 total time=   5.3s
[CV 2/5] END ..C=1, epsilon=0.1, kernel=linear;, score=-0.619 total time=   4.8s
[CV 3/5] END ..C=1, epsilon=0.1, kernel=linear;, score=-0.572 total time=   5.1s
[CV 4/5] END ..C=1, epsilon=0.1, kernel=linear;, score=-0.581 total time=   5.4s
[CV 5/5] END ..C=1, epsilon=0.1, kernel=linear;, score=-0.551 total time=   5.3s
[CV 1/5] END C=1, epsilon=0.001, kernel=linear;, score=-0.606 total time=  53.4s
[CV 2/5] END C=1, epsilon=0.001, kernel=linear;, score=-0.623 total time=  55.7s
[CV 3/5] END C=1, epsilon=0.001, kernel=linear;, score=-0.580 total time=  57.5s
[CV 4/5] END C=1, epsilon=0.001, kernel=linear;, score=-0.590 total time=  53.6s
[CV 5/5] END C=1, epsilon=0.001, kernel=linear;, score=-0.558 total time=  54.5s
[CV 1/5] END .C=10, epsil

In [93]:
def load_metrics_with_parameters(path):
    metrics_list = []

    for filename in os.listdir(path):
        if filename.endswith('.csv') and 'metrics' in filename:
            metrics_filepath = os.path.join(path, filename)
            metrics = pd.read_csv(metrics_filepath)

            parts = filename.replace('.csv', '').split('_')
            spatial_res = parts[-3]
            temporal_res = parts[-1]
            model_name = f'res{spatial_res}_{temporal_res}h'
            metrics['model'] = model_name

            params_filename = f'parameter_spatial_{spatial_res}_temporal_{temporal_res}.csv'
            params_path = os.path.join(path, params_filename)
            params = pd.read_csv(params_path)

            for key, value in params.items():
                metrics[key] = value.values[0]

            metrics_list.append(metrics)

    all_metrics = pd.concat(metrics_list, ignore_index=True)
    column_order = ['model', 'C', 'epsilon', 'kernel', 'mse', 'rmse', 'r2']
    all_metrics = all_metrics[column_order]
    return all_metrics

In [101]:
def find_best_model(metrics_with_params_df, criterion):
    if criterion == 'r2':
        best = metrics_with_params_df.loc[metrics_with_params_df['r2'].idxmax()]
    elif criterion == 'rmse':
        best = metrics_with_params_df.loc[metrics_with_params_df['rmse'].idxmin()]
    elif criterion == 'mse':
        best = metrics_with_params_df.loc[metrics_with_params_df['mse'].idxmin()]
    else:
        raise ValueError("Criterion not recognized. Use 'r2' or 'rmse' or 'mse'.")
    return best

In [102]:
def display_best_model(best_model, criterion):
    print(f'\nBest Model Based on {criterion}:')
    print(best_model.to_string())

In [103]:
models_path = 'data/models/'

all_metrics_with_params = load_metrics_with_parameters(models_path)

print('All Model Evaluation Metrics and Parameters:')
print(all_metrics_with_params.to_string())

All Model Evaluation Metrics and Parameters:
      model   C  epsilon kernel            mse        rmse        r2
0  res8_24h  10    0.001    rbf    7462.555452   86.386084  0.920221
1  res7_24h  10    0.001    rbf   45881.270617  214.199138  0.864278
2   res8_6h  10    0.100    rbf   65169.998297  255.284152  0.782421
3   res7_6h  10    0.001    rbf  121565.797962  348.662871  0.672934
4   res7_2h  10    0.100    rbf   23027.206354  151.747179  0.869236
5   res8_1h  10    0.100    rbf    2589.249702   50.884671  0.836425
6   res7_1h  10    0.100    rbf    6189.253861   78.671811  0.846672
7   res8_2h  10    0.100    rbf    7652.122396   87.476411  0.833087


In [107]:
criterion = 'rmse'
best_model = find_best_model(all_metrics_with_params, criterion)
display_best_model(best_model, criterion)


Best Model Based on rmse:
model          res8_1h
C                   10
epsilon            0.1
kernel             rbf
mse        2589.249702
rmse         50.884671
r2            0.836425


In [108]:
criterion = 'r2'
best_model = find_best_model(all_metrics_with_params, criterion)
display_best_model(best_model, criterion)


Best Model Based on r2:
model         res8_24h
C                   10
epsilon          0.001
kernel             rbf
mse        7462.555452
rmse         86.386084
r2            0.920221


In [110]:
spatial_res = 8
temporal_res = 1

hyperparameters = [{
        'kernel': ['rbf'],
        'epsilon': [0.1],
        'C': [10]
    }]

df_pred = create_prediction_dataset(df, temporal_res, spatial_res)
X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled,scaler = split_scale_data(df_pred)
model = gridsearch(X_train_scaled,y_train_scaled, temporal_res, spatial_res, hyperparameters)
model_evaluation(model, scaler, X_test_scaled, y_test_scaled, temporal_res, spatial_res)


Dataset prepared with 
spatial resolution: h3_res_8 
temporal resolution: 1_h

Starting grid search...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ....C=10, epsilon=0.1, kernel=rbf;, score=-0.420 total time=20.6min
[CV 2/5] END ....C=10, epsilon=0.1, kernel=rbf;, score=-1.863 total time=12.6min
[CV 3/5] END ....C=10, epsilon=0.1, kernel=rbf;, score=-0.458 total time=20.8min
[CV 4/5] END ....C=10, epsilon=0.1, kernel=rbf;, score=-0.461 total time=21.2min
[CV 5/5] END ....C=10, epsilon=0.1, kernel=rbf;, score=-1.011 total time=19.2min
Parameters successfully saved to data/models/parameter_spatial_8_temporal_1.csv
Best parameters: 
{'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
Metrics saved to data/models/metrics_spatial_8_temporal_1.csv
Evaluation metrics: 
{'mse': 1840.3858950592596, 'rmse': 42.8997190557148, 'r2': 0.8481705195220307}


In [ ]:
spatial_res = 8
temporal_res = 24

hyperparameters = [{
        'kernel': ['rbf'],
        'epsilon': [0.001],
        'C': [10]
    }]

df_pred = create_prediction_dataset(df, temporal_res, spatial_res)
X_train_scaled,y_train_scaled,X_test_scaled,y_test_scaled,scaler = split_scale_data(df_pred)
model = gridsearch(X_train_scaled,y_train_scaled, temporal_res, spatial_res, hyperparameters)
model_evaluation(model, scaler, X_test_scaled, y_test_scaled, temporal_res, spatial_res)


Dataset prepared with 
spatial resolution: h3_res_8 
temporal resolution: 24_h

Starting grid search...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..C=10, epsilon=0.001, kernel=rbf;, score=-0.410 total time=14.4min
[CV 2/5] END ..C=10, epsilon=0.001, kernel=rbf;, score=-1.942 total time=15.4min
[CV 3/5] END ..C=10, epsilon=0.001, kernel=rbf;, score=-0.460 total time=15.6min
[CV 4/5] END ..C=10, epsilon=0.001, kernel=rbf;, score=-0.402 total time=14.4min
[CV 5/5] END ..C=10, epsilon=0.001, kernel=rbf;, score=-0.999 total time=15.9min
